In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms
import os
import numpy as np
#from torchinfo import summary
from torch.utils.data.dataloader import DataLoader
#from torch.data import random_split
from torchvision import transforms

In [4]:
from Convs_Unet import UNet

In [5]:
import imgaug.augmenters as iaa

In [5]:
model = UNet(n_channels = 3 , n_classes = 1)
print("Loading Unet model")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device {device}")
model.to(device=device)
model.load_state_dict(torch.load("../Tiles_512_Sigmod6CT_ep150.pth", map_location=device))
print("model loaded")

Loading Unet model
Using device cuda
model loaded


In [4]:
#pip install torchinfo

In [23]:
x = model(torch.from_numpy(np.random.rand(1,3,300,300)).to(device = device, dtype = torch.float32) )

In [11]:
x.shape

torch.Size([1, 1, 300, 300])

In [21]:
for x in model.state_dict():
    print(x , model.state_dict()[x].shape)

inc.double_conv.0.weight torch.Size([64, 3, 3, 3])
inc.double_conv.0.bias torch.Size([64])
inc.double_conv.1.weight torch.Size([64])
inc.double_conv.1.bias torch.Size([64])
inc.double_conv.1.running_mean torch.Size([64])
inc.double_conv.1.running_var torch.Size([64])
inc.double_conv.1.num_batches_tracked torch.Size([])
inc.double_conv.3.weight torch.Size([64, 64, 3, 3])
inc.double_conv.3.bias torch.Size([64])
inc.double_conv.4.weight torch.Size([64])
inc.double_conv.4.bias torch.Size([64])
inc.double_conv.4.running_mean torch.Size([64])
inc.double_conv.4.running_var torch.Size([64])
inc.double_conv.4.num_batches_tracked torch.Size([])
down1.maxpool_conv.1.double_conv.0.weight torch.Size([128, 64, 3, 3])
down1.maxpool_conv.1.double_conv.0.bias torch.Size([128])
down1.maxpool_conv.1.double_conv.1.weight torch.Size([128])
down1.maxpool_conv.1.double_conv.1.bias torch.Size([128])
down1.maxpool_conv.1.double_conv.1.running_mean torch.Size([128])
down1.maxpool_conv.1.double_conv.1.running_va

In [26]:
print(model)

UNet(
  (inc): DoubleConv(
    (double_conv): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (down1): DownConv(
    (maxpool_conv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): DoubleConv(
        (double_conv): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (4): BatchNorm2d(128, eps=1e-05, mo

In [25]:
summary(model ,input_size = (1 ,3 ,50 ,50))

Layer (type:depth-idx)                        Output Shape              Param #
UNet                                          [1, 1, 50, 50]            --
├─DoubleConv: 1-1                             [1, 64, 50, 50]           --
│    └─Sequential: 2-1                        [1, 64, 50, 50]           --
│    │    └─Conv2d: 3-1                       [1, 64, 50, 50]           1,792
│    │    └─BatchNorm2d: 3-2                  [1, 64, 50, 50]           128
│    │    └─ReLU: 3-3                         [1, 64, 50, 50]           --
│    │    └─Conv2d: 3-4                       [1, 64, 50, 50]           36,928
│    │    └─BatchNorm2d: 3-5                  [1, 64, 50, 50]           128
│    │    └─ReLU: 3-6                         [1, 64, 50, 50]           --
├─DownConv: 1-2                               [1, 128, 25, 25]          --
│    └─Sequential: 2-2                        [1, 128, 25, 25]          --
│    │    └─MaxPool2d: 3-7                    [1, 64, 25, 25]           --
│    │    └

In [3]:
np.int = int
def expand_by_mirroring(image_train_raw,
                        len_row, 
                        len_col,
                        half_len): #half_len = 512//4

    """ 
# For testing,
    1. expand left and up side by GLOBAL['len_Unet_tile']/4.
    2. expand right and button side by "residue + GLOBAL['len_Unet_tile']/4", where residue equals "GLOBAL['len_Unet_tile'] - len_row%GLOBAL['len_Unet_tile']"

"""
    len_img_tile = 4*half_len
    if len_row%len_img_tile == 0:
        row_expand = np.int(len_row + 0.5 * len_img_tile)
        col_expand = np.int(len_col + 0.5 * len_img_tile)        
    else:
        row_expand = np.int(len_row + 1.5 * len_img_tile - len_row%len_img_tile)
        col_expand = np.int(len_col + 1.5 * len_img_tile - len_col%len_img_tile)
    
    img_expand = np.uint8(np.zeros([row_expand, col_expand, 3]))

    

    if len_row%len_img_tile == 0:
        img_expand[half_len:half_len + len_row, half_len : half_len + len_col, :] = image_train_raw  
        img_expand[:half_len, :, :] = np.flipud(img_expand[half_len : 2 * half_len, :, :])
        img_expand[half_len + len_row:row_expand, :, :] = np.flipud(img_expand[half_len + len_row - half_len:half_len + len_row, :, :])
        img_expand[:, :half_len, :] = np.fliplr(img_expand[:, half_len :  2 * half_len, :])
        img_expand[:, half_len + len_col:col_expand, :] = np.fliplr(img_expand[:, half_len + len_col - half_len:half_len + len_col, :])    
         
    else:
        img_expand[half_len:half_len + len_row, half_len : half_len + len_col, :] = image_train_raw  
        img_expand[:half_len, :, :] = np.flipud(img_expand[half_len : 2 * half_len, :, :])
        img_expand[half_len + len_row:row_expand, :, :] = np.flipud(img_expand[half_len + len_row - (5 * half_len - len_row%len_img_tile):half_len + len_row, :, :])
        img_expand[:, :half_len, :] = np.fliplr(img_expand[:, half_len :  2 * half_len, :])
        img_expand[:, half_len + len_col:col_expand, :] = np.fliplr(img_expand[:, half_len + len_col - (5 * half_len - len_col%len_img_tile):half_len + len_col, :])    
        
    return img_expand


In [6]:
def augmt5_0325(img_expand,
                aug_method):

    aug_dict = {"rt00":rt00(img_expand),
                "rt90":rt90(img_expand),
                "rt180":rt180(img_expand),
                "fplr":fplr(img_expand), 
                "fpud":fpud(img_expand)}
    image = aug_dict[aug_method]    
    
    return image

def rt00(img):
    img_aug = img
    return img_aug

def rt90(img):
    img_aug = np.rot90(img)
    return img_aug

def rt180(img):
    img_aug = np.rot90(img, 2)
    return img_aug

def fplr(img):
    img_aug = iaa.Fliplr(1.0)(image = img)
    return img_aug

def fpud(img):
    img_aug = iaa.Flipud(1.0)(image = img)
    return img_aug

In [12]:
dirs1 = os.listdir("../Data/Cropped/Im_inp/")
dirs2 = os.listdir("../Data/Cropped/Im_out/")
aug_list = ['rt00', 'rt90', 'rt180', 'fplr', 'fpud']
new_dir = "../Data/processed/"
for i , dir_ in enumerate(dirs1):
    new_img = Image.open("../Data/Cropped/Im_inp/" + dir_)
    new_img = np.array(new_img)[:,:,:3]
    print(new_img.shape)
    for aug in aug_list :
        new_img1 = augmt5_0325(new_img , aug)
        shape = np.array(new_img1).shape
        new_new_img1 = expand_by_mirroring(new_img1 , shape[0] , shape[1] ,512//4) 
        new_new_img1 = Image.fromarray(new_new_img1)
        new_new_img1.save(new_dir + "Im_inp/" + dir_.split('.')[0]+ "_" + aug + ".tif")

for i , dir_ in enumerate(dirs2):
    new_img = Image.open("../Data/Cropped/Im_out/" + dir_)
    new_img = np.array(new_img)[:,:,:3]
    print(new_img.shape)
    for aug in aug_list :
        new_img1 = augmt5_0325(new_img , aug)
        shape = np.array(new_img1).shape
        new_new_img1 = expand_by_mirroring(new_img1 , shape[0] , shape[1] , 512//4)
        new_new_img1 = Image.fromarray(new_new_img1)
        new_new_img1.save(new_dir + "Im_out/" + dir_.split('.')[0] + "_" + aug + ".tif")
    

(4500, 7500, 3)
(4500, 7500, 3)
(4500, 7500, 3)
(5100, 8500, 3)
(5100, 8500, 3)
(5100, 8500, 3)
(4500, 7500, 3)
(5100, 8500, 3)


In [13]:
def img_aug(img ,img_name, step_x , step_y ,  dim_x ,dim_y , save_loc = "../Data/Full_Aug/Im_inp/"):
    c = 0
    for i in range(0 ,img.shape[0] , step_x):
        if i + dim_x > img.shape[0]:
            break
        for j in range(0 , img.shape[1] , step_y):
            if j + dim_y > img.shape[1] :
                break 
            new_img = img[i : i + dim_x , j : j + dim_y ,:].copy()
            new_img = Image.fromarray(new_img)
            new_img.save(save_loc + img_name + f"_{i}_{j}" + ".tif")
            c+=1
    print("Done !!")
    print(f"Created {c} augmented images of {img_name} of size : {(dim_x ,dim_y)} taking the step size : {(step_x ,step_y)}")
            

In [14]:
dirs1 = os.listdir("../Data/processed/Im_inp/")
dirs2 = os.listdir("../Data/processed/Im_out/")

for i , name in enumerate(dirs1):
    img_aug(np.array(Image.open("../Data/processed/Im_inp/" + name)) , name.split(".")[0] , 100 , 100 ,512 ,512 ,"../Data/Full_Aug/Im_inp/" )
for i , name in enumerate(dirs2):
    img_aug(np.array(Image.open("../Data/processed/Im_out/" + name)) , name.split(".")[0] , 100 , 100 ,512 ,512 , "../Data/Full_Aug/Im_out/")

Done !!
Created 3300 augmented images of MC16_S1_0_fplr of size : (512, 512) taking the step size : (100, 100)
Done !!
Created 3300 augmented images of MC16_S1_0_fpud of size : (512, 512) taking the step size : (100, 100)
Done !!
Created 3300 augmented images of MC16_S1_0_rt00 of size : (512, 512) taking the step size : (100, 100)
Done !!
Created 3300 augmented images of MC16_S1_0_rt180 of size : (512, 512) taking the step size : (100, 100)
Done !!
Created 3300 augmented images of MC16_S1_0_rt90 of size : (512, 512) taking the step size : (100, 100)
Done !!
Created 3300 augmented images of MC16_S1_1_fplr of size : (512, 512) taking the step size : (100, 100)
Done !!
Created 3300 augmented images of MC16_S1_1_fpud of size : (512, 512) taking the step size : (100, 100)
Done !!
Created 3300 augmented images of MC16_S1_1_rt00 of size : (512, 512) taking the step size : (100, 100)
Done !!
Created 3300 augmented images of MC16_S1_1_rt180 of size : (512, 512) taking the step size : (100, 100)